In [1]:
import sqlite3
import gradio as gr
import yaml
import os

# Load configuration
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)

db_name = config["database"]["name"]
table_name = config["database"]["table"]

# SQL type mapping
def sql_type(field_type):
    return {
        "text": "TEXT",
        "number": "INTEGER",
        "checkbox": "BOOLEAN",
        "dropdown": "TEXT",
        "radio": "TEXT",
    }.get(field_type, "TEXT")

# Create table from config
def create_table():
    fields = []
    for tab in config["interface"]["tabs"]:
        for field in tab["fields"]:
            name = field["name"]
            ftype = sql_type(field["type"])
            fields.append(f"{name} {ftype}")
    with sqlite3.connect(db_name) as conn:
        conn.execute(
            f"CREATE TABLE IF NOT EXISTS {table_name} (id TEXT PRIMARY KEY, {', '.join(fields)})"
        )
        conn.commit()

# Build Gradio UI
def build_ui():
    

    with gr.Blocks() as demo:
        gr.Markdown("## YAML-Powered Dynamic Form")

        with gr.Row():
            id_input = gr.Textbox(label="Record ID", placeholder="Enter ID to save/load")
            load_btn = gr.Button("🔍 Load")
            save_btn = gr.Button("💾 Save")

        for tab in config["interface"]["tabs"]:
            with gr.Tab(tab["name"]):
                for field in tab["fields"]:
                    key = field["name"]
                    label = field.get("label", key)
                    default = field.get("default", None)
                    ftype = field["type"]

                    if ftype == "text":
                        comp = gr.Textbox(label=label, value=default)
                    elif ftype == "number":
                        comp = gr.Number(label=label, value=default)
                    elif ftype == "checkbox":
                        comp = gr.Checkbox(label=label, value=default)
                    elif ftype == "dropdown":
                        comp = gr.Dropdown(label=label, choices=field["options"], value=default)
                    elif ftype == "radio":
                        comp = gr.Radio(label=label, choices=field["options"], value=default)
                    else:
                        comp = gr.Textbox(label=label)

                    field_components[key] = comp

        status_box = gr.Textbox(label="Status", interactive=False)

        def load_data(record_id):
            if not record_id.strip():
                return [""] * len(field_components), "❌ Please enter a Record ID to load."
            print(record_id)

            with sqlite3.connect(db_name) as conn:
                cursor = conn.cursor()
                cursor.execute(f"SELECT * FROM {table_name} WHERE id = ?", (record_id,))
                row = cursor.fetchone()

                if row:
                    col_names = [description[0] for description in cursor.description]
                    values = [row[col_names.index(key)] if key in col_names else None for key in field_components]
                    return values+["✅ Data loaded."]
                else:
                    return [""] * len(field_components)+ ["⚠️ No record found. Showing defaults."]

        def save_data(record_id, *values):
            if not record_id.strip():
                return "❌ Please enter a Record ID before saving."

            with sqlite3.connect(db_name) as conn:
                cursor = conn.cursor()
                cursor.execute(f"SELECT 1 FROM {table_name} WHERE id = ?", (record_id,))
                if cursor.fetchone():
                    update_clause = ", ".join(f"{k}=?" for k in field_components)
                    cursor.execute(
                        f"UPDATE {table_name} SET {update_clause} WHERE id = ?",
                        (*values, record_id),
                    )
                else:
                    placeholders = ", ".join("?" for _ in field_components)
                    columns = ", ".join(field_components.keys())
                    cursor.execute(
                        f"INSERT INTO {table_name} (id, {columns}) VALUES (?, {placeholders})",
                        (record_id, *values),
                    )
                conn.commit()
            return "✅ Data saved successfully."

        load_btn.click(
            fn=load_data,
            inputs=[id_input],
            outputs=[*field_components.values(), status_box]
        )

        save_btn.click(
            fn=save_data,
            inputs=[id_input] + list(field_components.values()),
            outputs=status_box
        )

    return demo

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Initialize and run
field_components = {}
create_table()
app = build_ui()

In [ ]:
a+3

In [3]:
app.launch(server_name="0.0.0.0", server_port=7860)

* Running on local URL:  http://0.0.0.0:7860
* To create a public link, set `share=True` in `launch()`.


1
33
2
1


Traceback (most recent call last):
  File "/usr/local/lib/python3.11/site-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/gradio/blocks.py", line 2230, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/gradio/blocks.py", line 1957, in postprocess_data
    self.validate_outputs(block_fn, predictions)  # type: ignore
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/gradio/blocks.py", line 1912, in validate_outputs
    rai

1
2


Traceback (most recent call last):
  File "/usr/local/lib/python3.11/site-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/gradio/blocks.py", line 2230, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/gradio/blocks.py", line 1957, in postprocess_data
    self.validate_outputs(block_fn, predictions)  # type: ignore
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/gradio/blocks.py", line 1912, in validate_outputs
    rai

1
